In [ ]:
from source import dfs
from utils import run_model, summary_plots, to_scaled_df
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [ ]:
config={"model": "bisecting_kmeans","k": 3,"feature_names": ['avg_spent','number_txns','avg_duration','frequency', 'num_nftcontract', 'num_currency'] }

folder = "experiment"
custom_path = f"{folder}/model.parquet"
run_model(dfs, config, custom_path)

In [ ]:
conf = pyspark.SparkConf()
spark = SparkSession.builder.getOrCreate()
output = spark.read.parquet(custom_path)
scaled_df = to_scaled_df(output, config)

In [ ]:
exprs = {x: "mean" for x in scaled_df.columns}
scaled_df.groupBy("prediction").agg(exprs).show()

In [ ]:
out_dfp = scaled_df.toPandas()
out_dfp['prediction'] = pd.Categorical(out_dfp.prediction)
out_dfp
# summary_plots(out_dfp, config, folder)

In [10]:
import seaborn as sns

p = out_dfp.melt(id_vars='prediction', var_name="features")
print(p)

sns.catplot(data=p, x='value', y='prediction', estimator="mean", hue='prediction',col='features', col_wrap=1)
# sns.catplot(data=p, x='value', y='prediction', hue='prediction',col='features', col_wrap=1,kind='violin')
# sns.swarmplot(data=p, x='value', y='prediction', hue='prediction')

# g = sns.catplot(
#     data=out_dfp,
#     x="avg_spent",
#     y="prediction",
#     hue="prediction",
#     palette="husl",
#     kind="box"
# )